In [ ]:
print("********************Methods*********************************")

In [ ]:
def leveshteinDistance(s1, s2):
    """Computes the Levenshtein distance between two arrays (strings too).
    Such distance is the minimum number of operations needed to transform one array into
    the other, where an operation is an insertion, deletion, or substitution of a single
    item (like a char). This implementation (Wagner-Fischer algorithm with just 2 lines)
    uses O(min(|s1|, |s2|)) space.
    """
    if s1 == s2: return 0
    if len(s1) > len(s2):
        s1, s2 = s2, s1
    r1 = list(range(len(s2) + 1))
    r2 = [0] * len(r1)
    i = 0
    for c1 in s1:
        r2[0] = i + 1
        j = 0
        for c2 in s2:
            if c1 == c2:
                r2[j+1] = r1[j]
            else:
                a1 = r2[j]
                a2 = r1[j]
                a3 = r1[j+1]
                if a1 > a2:
                    if a2 > a3:
                        r2[j+1] = 1 + a3
                    else:
                        r2[j+1] = 1 + a2
                else:
                    if a1 > a3:
                        r2[j+1] = 1 + a3
                    else:
                        r2[j+1] = 1 + a1
            j += 1
        aux = r1; r1 = r2; r2 = aux
        i += 1
    return r1[-1]

In [ ]:
'''if both seq are the same the score will be 0'''
def rankDistance(seq1,seq2):
    score=0
    seq1_arr=list(seq1)
    seq2_arr=list(seq2)

    for i in range(len(seq1_arr)):
        index=seq2_arr.index(seq1_arr[i]) if seq1_arr[i] in seq2_arr else 0
        score=score+abs(i-index)
        seq1_arr[i]='*'
        seq2_arr[index]='*'
    if(seq1_arr==seq2_arr):
        for i in range(len(seq1_arr)):
            j= i if seq1_arr[i]!='*' else 0
            score=score+j
            k= -i if seq2_arr[i]!='*' else 0
            score=score+j
    else:
        for i in range(len(seq1_arr)):
            j= i if seq1_arr[i]!='*' else 0
            score=score+j
        for i in range(len(seq2_arr)):
            j= i if seq2_arr[i]!='*' else 0
            score=score+j
    return score

In [ ]:
#localAlighnment
'''
for the calculation is used bio python
gap_open_penalty and gap_extend_penalty neeed to have negative value
'''
def local_alighn_score(seq1,seq2,gap_open_penalty,gap_extend_penalty):
    from Bio import pairwise2
    from Bio.SubsMat.MatrixInfo import blosum62
    a = pairwise2.align.localds(seq1, seq2, blosum62, gap_open_penalty,gap_extend_penalty)
    (s1,s2,score,start,end) = a[0]
    gaps=s1.count("-")+s2.count("-")
    return score,gaps;

In [ ]:
#globalAlighnment
'''
for the calculation is used bio python
gap_open_penalty and gap_extend_penalty neeed to have negative value
'''
def global_alighn_score(seq1,seq2,gap_open_penalty,gap_extend_penalty):
    from Bio.SubsMat import MatrixInfo as matlist
    from Bio import pairwise2
    matrix = matlist.blosum62
    a = pairwise2.align.globalds(seq1,seq2,matrix,gap_open_penalty,gap_extend_penalty)
    (s1,s2,score,start,end) = a[0]
    gaps=s1.count("-")+s2.count("-")
    return score,gaps;

In [ ]:
def similarity(global_alighn_score,local_alighn_score):
    if(global_alighn_score > local_alighn_score):
        return global_alighn_score
    else:
        return local_alighn_score

In [ ]:
def calculation(N,ratioComp): 
    import math
    import time
    from functools import reduce
    mean=ratioComp.similarity.sum()/N
    sum1=reduce(lambda x: (x -mean) / 2, ratioComp.similarity)
    stand_dev=math.sqrt(pow(sum1, 2)/N)
    signalToNoise=mean/stand_dev
    for x in ratioComp:
        x.mean=mean
        x.standart_dev=standart_dev
        x.signalToNoise=signalToNoise

In [ ]:
'''The center star method'''

In [ ]:
def substitutionCost(a, b):
    import numpy
    import pandas
    """Return the cost of substituting character a with character b"""
    return 0 if a == b else 3

def calculateCostMatrix(a, b, gapCost=2):
    import numpy
    import pandas
    """For two sequences a, b, calculate their edit distance matrix"""
    #Initialize n+1 x m+1 matrix. x axis <--> a, y axis <--> b
    xdim = len(b) + 1
    ydim = len(a) + 1
    matrix = numpy.zeros((xdim, ydim))
    #Initialize first row and first col with n*gap cost
    matrix[0,:] = numpy.arange(0, gapCost*ydim, step=gapCost)
    matrix[:,0] = numpy.arange(0, gapCost*xdim, step=gapCost)
    #Fill array
    for y in range(1, ydim):
        for x in range(1, xdim):
            substCost = matrix[x-1, y-1] + substitutionCost(b[x-1], a[y-1])
            matrix[x,y] = min(substCost, matrix[x-1, y] + gapCost, matrix[x, y-1] + gapCost)
    return matrix

def getCentralStarDistance(a, b, gapCost=2):
    import numpy
    import pandas
    """Get the edit distance of two sequences a and b"""
    return calculateCostMatrix(a, b, gapCost)[-1,-1]

In [ ]:
#Regressor to predict global alighnement score

In [ ]:
'''naighbourhood count'''
def word_neighborhood_count_score(seq1, seq2):
    score = 0
    W=[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]
    # for each base in the match
    for i in range(0,len(seq1)):
        counter=0
        start=i
        end=i+9
        seq1_len=len(seq1)-1
        seq2_len=len(seq2)-1
        while start < end:
            if(start >= seq1_len or start >= seq2_len):
                score = score - W[counter]
            elif(seq1[i] == seq2[i+counter]):
                score = score + W[counter] 
                break
            else:
                score = score - W[counter]
            counter = counter + 1
            start=start+1
    return score

In [ ]:
from difflib import SequenceMatcher 
  
def longestSubstring(str1,str2): 
    seqMatch = SequenceMatcher(None,str1,str2) 
    match = seqMatch.find_longest_match(0, len(str1), 0, len(str2)) 
    return match.size

In [ ]:
print("****************Data Structures******************")

In [ ]:
class DataSetInfo:
    def __init__(self,first_org_name,second_org_name,chunk_index_first_org,chunk_index_second_org,first_chunk_value,second_chunk_value,
                global_alighn_score,global_alighn_gap,local_alighn_score,local_alighn_gap,similarity,
                 word_count_score,longest_seq,rank_distance,levenshtein_distance,central_star_distance):
        self.first_org_name = first_org_name
        self.second_org_name = second_org_name
        self.chunk_index_first_org = chunk_index_first_org
        self.chunk_index_second_org  = chunk_index_second_org
        self.first_chunk_value = first_chunk_value
        self.second_chunk_value = second_chunk_value
        #alighnment based
        self.global_alighn_score = global_alighn_score
        self.global_alighn_gap=global_alighn_gap
        self.local_alighn_score = local_alighn_score
        self.local_alighn_gap=local_alighn_gap
        self.similarity = similarity
        #alignment_free
        self.word_count_score = word_count_score
        self.longest_seq = longest_seq 
        self.rank_distance = rank_distance 
        self.levenshtein_distance = levenshtein_distance
        self.central_star_distance=central_star_distance
        #other metrics
        self.mean = None 
        self.standart_dev = None
        self.signalToNoise = None

    def as_dict(self):
        return {'org1': self.first_org_name, 'org2': self.second_org_name, 'chunk_or1': self.first_chunk_value,
               'chunk_or2': self.second_chunk_value,'global_align': self.global_alighn_score,
                'global_alighn_gap':self.global_alighn_gap,'local_align': self.local_alighn_score,
                'local_alighn_gap':self.local_alighn_gap,'similarity':self.similarity,
                'word_count_score':self.word_count_score,'longest_seq':self.longest_seq,
                'rank_distance':self.rank_distance,'levenshtein_distance':self.levenshtein_distance,
                'central_star_distance':self.central_star_distance,
               'mean': self.mean,'STD':self.standart_dev,'signalToNoise':self.signalToNoise}


In [ ]:
print("****************Pre-Processing******************")

In [ ]:
#Import Data
from Bio import SeqIO
sequence_virus_hiv = ""
for seq_record in SeqIO.parse("HIV.fasta", "fasta"):
    sequence_virus_hiv=sequence_virus_hiv+seq_record.seq
sequence_virus_covid= ""
for seq_record in SeqIO.parse("coronavirus.fasta", "fasta"):
    print(seq_record.seq)
    sequence_virus_covid=sequence_virus_covid+seq_record.seq
sequence_virus_tuberculosis= ""
for seq_record in SeqIO.parse("tuberculosis.fasta", "fasta"):
    sequence_virus_tuberculosis=sequence_virus_tuberculosis+seq_record.seq
sequence_virus_flu= ""
for seq_record in SeqIO.parse("influenca.fasta", "fasta"):
    sequence_virus_flu=sequence_virus_flu+seq_record.seq
sequence_virus_sars= ""
for seq_record in SeqIO.parse("SARS-CoV-2.fasta", "fasta"):
    sequence_virus_sars=sequence_virus_sars+seq_record.seq

In [ ]:
'''sliding windows to split data in k-mers'''
def split_to_k_mers(seq,step,window):
    result=[]
    for i in range(0,(len(seq)-window),step):
        result.append(seq[i:i + window])
    return result

In [ ]:
k=60
chunks_seq_virus_hiv=[sequence_virus_hiv[i:i+k] for i in range(0, len(sequence_virus_hiv), k)]
chunks_seq_virus_covid=[sequence_virus_covid[i:i+k] for i in range(0, len(sequence_virus_covid), k)]
chunks_seq_virus_tubercolosis=[sequence_virus_tuberculosis[i:i+k] for i in range(0, len(sequence_virus_tuberculosis), k)]

In [ ]:
k=60
chunks_seq_virus_flu=[sequence_virus_flu[i:i+k] for i in range(0, len(sequence_virus_flu), k)]
chunks_seq_virus_sars=[sequence_virus_sars[i:i+k] for i in range(0, len(sequence_virus_sars), k)]


In [ ]:
def chunk_preprocessing(org1Name,Org2Name,chunks_seq_virus1,chunks_seq_virus2,seq1
                                       ,seq2,gap_open_penalty,gap_extend_penalty,result):
    global_alighnment ,global_alighn_gaps= global_alighn_score(seq1,seq2,gap_open_penalty,gap_extend_penalty)
    local_alighnment,local_alighn_gaps= local_alighn_score(seq1,seq2,gap_open_penalty,gap_extend_penalty)
    sim_score = similarity(global_alighnment,local_alighnment)  
    
    word_count=word_neighborhood_count_score(seq1, seq2)
    central_star_distance=getCentralStarDistance(seq1, seq2)
    rank_distance=rankDistance(seq1, seq2)
    longest_substring=longestSubstring(seq1, seq2)
    leveshtein_distance=leveshteinDistance(seq1, seq2)
    
    
    info =  DataSetInfo(org1Name,Org2Name,chunks_seq_virus1,chunks_seq_virus2,seq1,seq2,
                        global_alighnment,global_alighn_gaps,local_alighnment,local_alighn_gaps,sim_score,
                        word_count,longest_substring,rank_distance,leveshtein_distance,central_star_distance)
    #global_alighnment,global_alighn_gaps,local_alighnment,local_alighn_gaps,sim_score,
    result.put(info)

In [ ]:
import threading, queue
  
def paraller_preprocessing(org1Name,Org2Name,chunks_seq_virus1,chunks_seq_virus2,gap_open_penalty,gap_extend_penalty):
    q = queue.Queue()
    threads = []
    for x in range(int(len(chunks_seq_virus1))):
        for y in range(int(len(chunks_seq_virus2))):
            seq1 = chunks_seq_virus1[x]
            seq2 = chunks_seq_virus2[y]
            t=threading.Thread(target=chunk_preprocessing, args=(org1Name,Org2Name,chunks_seq_virus1,chunks_seq_virus2,seq1,seq2,gap_open_penalty,gap_extend_penalty,q))
            threads.append(t)
            t.start()
     # Wait for all threads to finish
    for x in threads:
        x.join()
    result=list(q.queue)
    return result

In [ ]:
# import time
# start = time.time()
# result=paraller_preprocessing("HIV","Tubercolosis",chunks_seq_virus_hiv,chunks_seq_virus_tubercolosis,-10,-1)
# #tuberculosis
# end = time.time()
# print("Time for executing model")
# print(end - start)
# print(" in seconds")

In [ ]:
!pip install ml_metrics
import sklearn.metrics as metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from ml_metrics import rmse
import numpy as np

def regression_results(y_true, y_pred):# Regression metrics
    explained_variance=explained_variance_score(y_true, y_pred)
    mae=mean_absolute_error(y_true, y_pred) 
    mse=mean_squared_error(y_true, y_pred) 
    r2=r2_score(y_true, y_pred)
    print('explained_variance: ', round(explained_variance,4))    
    print('r2: ', round(r2,4))
    print('MAE: ', round(mae,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))

In [ ]:
!pip install progressbar
!pip install pandas
import pandas as pd
import progressbar
from time import sleep
bar = progressbar.ProgressBar(maxval=1000, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
l=int(len(chunks_seq_virus_tubercolosis)/1000)

bar.start()
for i in range(910, 1000):
    bar.update(i+1)
    result=paraller_preprocessing("Covid","HIV",chunks_seq_virus_covid,chunks_seq_virus_tubercolosis[i*l:(i+1)*l],-10,-1)
    dataFrame = pd.DataFrame([x.as_dict() for x in result])
    filename="data_Covid_HIV_real"+str(i)+".csv"
    dataFrame.to_csv(filename,index = False, header=True)
bar.finish()

In [ ]:
import sklearn
import joblib
print('The sklearn version is {}.'.format(sklearn.__version__))
print('The joblib version is {}.'.format(joblib.__version__))

In [ ]:
#merge data
import os
import glob
import pandas as pd


#all_filename_HT = [i for i in glob.glob('data_HIV_Tubercolosis*.{}'.format('csv'))]
#combined_csv_HT = pd.concat([pd.read_csv(f) for f in all_filenamesHT ])
all_filename_CT = [i for i in glob.glob('data_Covid_Tubercolosis*.{}'.format('csv'))]
combined_csv_CT = pd.concat([pd.read_csv(f) for f in all_filename_CT ])
data3=pd.read_csv("tuberculasis_HIV.csv")
data1= pd.read_csv("data_SARS_Flu.csv")
data2= pd.read_csv("data_HIV_Flu.csv")
data_hiv_hiv=pd.read_csv("data_HIV_HIV.csv")
df = pd.concat([data1,data2,data3,combined_csv_CT,data_hiv_hiv])
data_hiv_covid=pd.read_csv("data_HIV_Covid19.csv")
#combined_csv_CT.to_csv( "tuberculasis_covid.csv", index=False, encoding='utf-8-sig')

In [ ]:
df_data=df.drop(['org1', 'org2','mean', 'STD','signalToNoise', 'similarity'], axis=1)
data=df_data.dropna()
data.head(5)

In [ ]:
#Local Alignn GAp Predictor- 'local_alighn_gap'

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
scaler = StandardScaler() 
gap_pred_data=data.drop(['local_align','global_align', 'global_alighn_gap'], axis=1)
X=gap_pred_data.copy()
y=gap_pred_data['local_alighn_gap']
X_trainBase, X_testBase, y_train, y_test = train_test_split(X, y,test_size=0.2, shuffle=False)
X_train=X_trainBase.drop(['chunk_or1','chunk_or2','local_alighn_gap'], axis=1)
X_train=X_train.astype('int32')
X_train =scaler.fit_transform(X_train)
    #X_train = scaler.transform(X_train)
X_test=X_testBase.drop(['chunk_or1','chunk_or2','local_alighn_gap'], axis=1)
X_test=X_test.astype('int32')
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor()
param_grid = {'hidden_layer_sizes': [i for i in range(1,15)],
              'activation': ['relu'],
              'solver': ['adam'],
              'learning_rate': ['constant'],
              'learning_rate_init': [0.001],
              'power_t': [0.5],
              'alpha': [0.0001],
              'max_iter': [1000],
              'early_stopping': [False],
              'warm_start': [False]}
grid = GridSearchCV(mlp, param_grid=param_grid, verbose=True)
grid.fit(X_train, y_train)
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
print("\n The best score across ALL searched params:\n", grid.best_score_)
print("\n The best parameters across ALL searched params:\n", grid.best_params_)

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

regr = MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=11, learning_rate='constant',
             learning_rate_init=0.001, max_iter=1000, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False).fit(X_train, y_train)
y_pred = regr.predict(X_test)

In [ ]:
!pip install xgboost
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
model = XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07, 0.1], #so called `eta` value
              'max_depth': [5, 6, 7,8],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [200,300,500,600,700]}

grid = GridSearchCV(model,parameters, cv=2,n_jobs = 5,
                        verbose=True)
grid.fit(X_train, y_train)
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
print("\n The best score across ALL searched params:\n", grid.best_score_)
print("\n The best parameters across ALL searched params:\n", grid.best_params_)

In [ ]:
from xgboost import XGBRegressor
xgb_model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=5,
             min_child_weight=4, monotone_constraints='()',
             n_estimators=600, n_jobs=4, nthread=4, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, silent=1, subsample=0.7, tree_method='exact',
             validate_parameters=1, verbosity=None)

xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
regression_results(y_test, y_pred)

In [ ]:
from sklearn.linear_model import LinearRegression

reg_local_alighn_gap = LinearRegression({'copy_X': True, 'fit_intercept': False, 'normalize': True}).fit(X_train, y_train)
y_pred=reg_local_alighn_gap.predict(X_test)
regression_results(y_test, y_pred)
reg_local_alighn_gap.fit(X_test,y_test)

In [ ]:
!pip install joblib
import joblib
# save the model to disk
filename = 'local_aligh_gap_predictor.sav'
joblib.dump(reg_local_alighn_gap, filename)
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.predict(X_test)
regression_results(y_test, result)

In [ ]:
#Local Alighn predictor

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
scaler = StandardScaler() 
gap_pred_data=data.drop(['global_alighn_gap','global_align'], axis=1)
X=gap_pred_data.copy()
y=gap_pred_data['local_align']
X_trainBase, X_testBase, y_train, y_test = train_test_split(X, y,test_size=0.2, shuffle=False)
X_train_la=X_trainBase.drop(['chunk_or1','chunk_or2','local_align','local_alighn_gap'], axis=1)
#X_train_la['local_alighn_gap_pred']=reg_local_alighn_gap.predict(X_train)
X_train_la=X_train_la.astype('int32')
X_train_la =scaler.fit_transform(X_train_la)
    #X_train = scaler.transform(X_train)
X_test_la=X_testBase.drop(['chunk_or1','chunk_or2','local_align','local_alighn_gap'], axis=1)
#X_test_la['local_alighn_gap_pred']=reg_local_alighn_gap.predict(X_test)
X_test_la=X_test_la.astype('int32')
X_test_la = scaler.transform(X_test_la)

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor()
param_grid = {'hidden_layer_sizes': [i for i in range(1,15)],
              'activation': ['relu'],
              'solver': ['adam'],
              'learning_rate': ['constant'],
              'learning_rate_init': [0.001],
              'power_t': [0.5],
              'alpha': [0.0001],
              'max_iter': [1000],
              'early_stopping': [False],
              'warm_start': [False]}
grid = GridSearchCV(mlp, param_grid=param_grid, verbose=True)
grid.fit(X_train_la, y_train)
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
print("\n The best score across ALL searched params:\n", grid.best_score_)
print("\n The best parameters across ALL searched params:\n", grid.best_params_)

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

regr = MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=11, learning_rate='constant',
             learning_rate_init=0.001, max_iter=1000, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False).fit(X_train_la, y_train)
y_pred = regr.predict(X_test_la)
regression_results(y_test, y_pred)

In [ ]:
!pip install xgboost
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
model = XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07, 0.1], #so called `eta` value
              'max_depth': [5, 6, 7,8],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [200,300,500,600,700]}

grid = GridSearchCV(model,parameters, cv=2,n_jobs = 5,
                        verbose=True)
grid.fit(X_train_la, y_train)
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
print("\n The best score across ALL searched params:\n", grid.best_score_)
print("\n The best parameters across ALL searched params:\n", grid.best_params_)

In [ ]:
from xgboost import XGBRegressor
xgb_model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=5,
             min_child_weight=4, monotone_constraints='()',
             n_estimators=600, n_jobs=4, nthread=4, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, silent=1, subsample=0.7, tree_method='exact',
             validate_parameters=1, verbosity=None)

xgb_model.fit(X_train_la, y_train)
y_pred = xgb_model.predict(X_test_la)
regression_results(y_test, y_pred)

In [ ]:
!pip install joblib
from sklearn.linear_model import LinearRegression
import joblib
# save the model to disk
filename = 'local_aligh_predictor.sav'
#joblib.dump(reg_local_alighn, filename)
# load the model from disk
loaded_model = joblib.load(filename)
# result = loaded_model.predict(X_test_la)
# regression_results(y_test, result)

In [ ]:
loaded_model.fit(X_train_la,y_train)
result = loaded_model.predict(X_test_la)
regression_results(y_test, result)

In [ ]:
from sklearn.linear_model import LinearRegression

reg_local_alighn = LinearRegression({'copy_X': True, 'fit_intercept': False, 'normalize': True}).fit(X_train_la, y_train)
y_pred=reg_local_alighn.predict(X_test_la)
regression_results(y_test, y_pred)
reg_local_alighn.fit(X_test_la,y_test)

In [ ]:
# !pip install joblib
# import joblib
# # save the model to disk
# filename = 'local_aligh_predictor.sav'
# joblib.dump(reg_local_alighn, filename)
# # load the model from disk
# loaded_model = joblib.load(filename)
# result = loaded_model.predict(X_test_la)
# regression_results(y_test, result)

In [ ]:
#Global_alighn_gap predictor

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
scaler = StandardScaler() 
gap_pred_data=data.drop(['local_align','local_alighn_gap','global_align'], axis=1)
X=gap_pred_data.copy()
y=gap_pred_data['global_alighn_gap']
X_trainBase, X_testBase, y_train, y_test = train_test_split(X, y,test_size=0.2, shuffle=False)
X_train_ga_gap=X_trainBase.drop(['chunk_or1','chunk_or2','global_alighn_gap'], axis=1)
X_train_ga_gap=X_train_ga_gap.astype('int32')
X_train_ga_gap =scaler.fit_transform(X_train_ga_gap)
    #X_train = scaler.transform(X_train)
X_test_ga_gap=X_testBase.drop(['chunk_or1','chunk_or2','global_alighn_gap'], axis=1)
X_test_ga_gap=X_test_ga_gap.astype('int32')
X_test_ga_gap = scaler.transform(X_test_ga_gap)

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor()
param_grid = {'hidden_layer_sizes': [i for i in range(1,15)],
              'activation': ['relu'],
              'solver': ['adam'],
              'learning_rate': ['constant'],
              'learning_rate_init': [0.001],
              'power_t': [0.5],
              'alpha': [0.0001],
              'max_iter': [1000],
              'early_stopping': [False],
              'warm_start': [False]}
grid = GridSearchCV(mlp, param_grid=param_grid, verbose=True)
grid.fit(X_train_ga_gap, y_train)
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
print("\n The best score across ALL searched params:\n", grid.best_score_)
print("\n The best parameters across ALL searched params:\n", grid.best_params_)

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

regr = MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=11, learning_rate='constant',
             learning_rate_init=0.001, max_iter=1000, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False).fit(X_train_ga_gap, y_train)
y_pred = regr.predict(X_test_ga_gap)
regression_results(y_test, y_pred)

In [ ]:
!pip install xgboost
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
model = XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07, 0.1], #so called `eta` value
              'max_depth': [5, 6, 7,8],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [200,300,500,600,700]}

grid = GridSearchCV(model,parameters, cv=2,n_jobs = 5,
                        verbose=True)
grid.fit(X_train_ga_gap, y_train)
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
print("\n The best score across ALL searched params:\n", grid.best_score_)
print("\n The best parameters across ALL searched params:\n", grid.best_params_)

In [ ]:
from xgboost import XGBRegressor
xgb_model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=5,
             min_child_weight=4, monotone_constraints='()',
             n_estimators=600, n_jobs=4, nthread=4, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, silent=1, subsample=0.7, tree_method='exact',
             validate_parameters=1, verbosity=None)

xgb_model.fit(X_train_ga_gap, y_train)
y_pred = xgb_model.predict(X_test_ga_gap)
regression_results(y_test, y_pred)

In [ ]:
from sklearn.linear_model import LinearRegression

reg_global_alighn_gap = LinearRegression({'copy_X': True, 'fit_intercept': False, 'normalize': True}).fit(X_train_ga_gap, y_train)
y_pred=reg_global_alighn_gap.predict(X_test_ga_gap)
regression_results(y_test, y_pred)
reg_global_alighn_gap.fit(X_test_ga_gap,y_test)

In [ ]:
!pip install joblib
import joblib
# save the model to disk
filename = 'global_aligh_gap_predictor.sav'
joblib.dump(reg_global_alighn_gap, filename)
# load the model from disk
loaded_model_ga_gap = joblib.load(filename)
loaded_model_ga_gap.fit(X_train_ga_gap,y_train)
result = loaded_model_ga_gap.predict(X_test_ga_gap)
regression_results(y_test, result)

In [ ]:
#Global_alighn predictor

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
scaler = StandardScaler() 
gap_pred_data=data.drop(['local_align','local_alighn_gap','global_alighn_gap'], axis=1)
X=gap_pred_data.copy()
y=gap_pred_data['global_align']
X_trainBase, X_testBase, y_train, y_test = train_test_split(X, y,test_size=0.2, shuffle=False)
X_train_ga=X_trainBase.drop(['chunk_or1','chunk_or2','global_align'], axis=1)
X_train_ga['local_alighn_pred']=loaded_model.predict(X_train_ga)
temp=X_trainBase.drop(['chunk_or1','chunk_or2','global_align'], axis=1)
X_train_ga['global_alighn_gap_pred']=loaded_model_ga_gap.predict(temp)
X_train_ga=X_train_ga.astype('int32')
X_train_ga =scaler.fit_transform(X_train_ga)
    #X_train = scaler.transform(X_train)
X_test_ga=X_testBase.drop(['chunk_or1','chunk_or2','global_align'], axis=1)
X_test_ga['local_alighn_pred']=loaded_model.predict(X_test_ga)
temp=X_testBase.drop(['chunk_or1','chunk_or2','global_align'], axis=1)
X_test_ga['global_alighn_gap_pred']=loaded_model_ga_gap.predict(temp)
X_test_ga=X_test_ga.astype('int32')
X_test_ga = scaler.transform(X_test_ga)

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor()
param_grid = {'hidden_layer_sizes': [i for i in range(1,15)],
              'activation': ['relu'],
              'solver': ['adam'],
              'learning_rate': ['constant'],
              'learning_rate_init': [0.001],
              'power_t': [0.5],
              'alpha': [0.0001],
              'max_iter': [1000],
              'early_stopping': [False],
              'warm_start': [False]}
grid = GridSearchCV(mlp, param_grid=param_grid, verbose=True)
grid.fit(X_train_ga, y_train)
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
print("\n The best score across ALL searched params:\n", grid.best_score_)
print("\n The best parameters across ALL searched params:\n", grid.best_params_)

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

regr = MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=11, learning_rate='constant',
             learning_rate_init=0.001, max_iter=1000, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False).fit(X_train_ga, y_train)
y_pred = regr.predict(X_test_ga)
regression_results(y_test, y_pred)

In [ ]:
!pip install xgboost
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
model = XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07, 0.1], #so called `eta` value
              'max_depth': [5, 6, 7,8],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [200,300,500,600,700]}

grid = GridSearchCV(model,parameters, cv=2,n_jobs = 5,
                        verbose=True)
grid.fit(X_train_ga, y_train)
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
print("\n The best score across ALL searched params:\n", grid.best_score_)
print("\n The best parameters across ALL searched params:\n", grid.best_params_)

In [ ]:
!pip install xgboost
from xgboost import XGBRegressor
reg_model_ga= XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=5,
             min_child_weight=4, monotone_constraints='()',
             n_estimators=600, n_jobs=4, nthread=4, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, silent=1, subsample=0.7, tree_method='exact',
             validate_parameters=1, verbosity=None).fit(X_train_ga, y_train)
y_pred=reg_model_ga.predict(X_test_ga)
regression_results(y_test, y_pred)

In [ ]:
!pip install joblib
import joblib
!pip install xgboost
from xgboost import XGBRegressor
# save the model to disk
filename = 'global_aligh_predictor.sav'
joblib.dump(reg_model_ga, filename)
# load the model from disk
loaded_model_ga = joblib.load(filename)
loaded_model_ga.fit(X_train_ga,y_train)
result = loaded_model_ga.predict(X_test_ga)
regression_results(y_test, result)

In [ ]:
#**********************************Clustering start**********************************

In [ ]:
def nb_cluster(labels_true, labels_pred,X_test):
    import sklearn
    ami_score=sklearn.metrics.adjusted_mutual_info_score(labels_true, labels_pred)
    compl_score=sklearn.metrics.completeness_score(labels_true, labels_pred)
    adj_rand_score=sklearn.metrics.adjusted_rand_score(labels_true, labels_pred)
    calinski_harabasz_score=sklearn.metrics.calinski_harabasz_score(X_test, labels_pred)
    davies_bouldin_score=sklearn.metrics.davies_bouldin_score(X_test, labels_pred)
    fowlkes_mallows_score=sklearn.metrics.fowlkes_mallows_score(labels_true, labels_pred)
    homogenity_score=sklearn.metrics.homogeneity_score(labels_true, labels_pred)
    mutual_info_score=sklearn.metrics.mutual_info_score(labels_true, labels_pred)
    silhouette_score=sklearn.metrics.silhouette_score(X_test, labels_pred)
    print("**************************")
    print("ami_score: {}".format(ami_score))
    print("compl_score: {}".format(compl_score))
    print("adj_rand_score: {}".format(adj_rand_score))
    print("calinski_harabasz_score: {}".format(calinski_harabasz_score))
    print("davies_bouldin_score: {}".format(davies_bouldin_score))
    print("fowlkes_mallows_score: {}".format(fowlkes_mallows_score))
    print("homogenity_score: {}".format(homogenity_score))
    print("mutual_info_score: {}".format(mutual_info_score))
    print("silhouette_score: {}".format(silhouette_score))

In [ ]:
def map_data_to_clusters(X_train,X_test):
    from sklearn.cluster import KMeans
    cluster = KMeans(n_clusters=15, init='k-means++', random_state=0)
    cluster.fit_predict(X_train)
    return cluster.predict(X_test),cluster

In [ ]:
data_hiv_covid

In [ ]:
dataFrame

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

train_set=data.copy()
train_set=train_set.drop(['chunk_or1','chunk_or2','local_align','local_alighn_gap','global_alighn_gap','global_align'], axis=1)
#train_set_transform =scaler.fit_transform(train_set)
train_set['local_alighn_pred']=data['local_align']
train_set['global_alighn_gap_pred']=data['global_alighn_gap']
train_set['global_alighn_pred']=data['global_align']
train_set=train_set.dropna()
#loaded_model_ga.predict(train_set_transform)

scaler=StandardScaler()
test_set=dataFrame.copy()
test_set=test_set.drop(['org1', 'org2','chunk_or1','chunk_or2','mean', 'STD','signalToNoise', 'similarity','local_align','local_alighn_gap','global_alighn_gap','global_align'], axis=1)
test_set=test_set.dropna()
test_set=test_set.astype('int32')
test_set_transform =scaler.fit_transform(test_set)
test_set['local_alighn_pred']=loaded_model.predict(test_set_transform)
test_set['global_alighn_gap_pred']=loaded_model_ga_gap.predict(test_set_transform)
test_set_transform =scaler.fit_transform(test_set)
test_set['global_alighn_pred']=loaded_model_ga.predict(test_set_transform)

In [ ]:
#check which you need and which you don`t
!pip install pyeasyga
import time
from pyeasyga import pyeasyga
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import MeanShift
train_data = pd.DataFrame(train_set, columns=['global_alighn_pred'])

In [ ]:
start = time.time()
clustering_free = MeanShift()
cluster_labels_free = clustering.fit_predict(train_data)
print("Best cluster num is :", len(list(set(cluster_labels_free))))
silhouette_avg = silhouette_score(df1, cluster_labels_free)
print("The average silhouette_score is :", silhouette_avg)
end = time.time()
print("Time for executing mean shift model")
print(end - start)
print(" in seconds")

In [ ]:
def fitness(individual, data):
    temp=0;
    cNum=1
    for x in data:
        if temp < x['score']:
            temp=x['score']
            cNum=x['clusterNum']
    return cNum


In [ ]:
start = time.time()
#KMeans++
data=[];
for i in range(2,20):
    cluster_labels = KMeans(n_clusters=i, init='k-means++').fit_predict(train_data)
    data.append({'clusterNum': i , 'score': silhouette_score(train_data, cluster_labels)})
ga = pyeasyga.GeneticAlgorithm(data)
ga.fitness_function = fitness               # set the GA's fitness function
ga.run()                                    # run the GA
print(ga.best_individual())
end = time.time()
print("Time for executing Kmeans++ with GA")
print(end - start)
print(" in seconds")

In [ ]:
from sklearn.cluster import AffinityPropagation
import numpy as np

start = time.time()
clustering = AffinityPropagation().fit_predict(train_data)
print("Best cluster num is :", len(list(set(clustering))))
print('score '+ str(silhouette_score(train_data, clustering)))
end = time.time()
print("Time for executing Kmeans++ with GA")
print(end - start)
print(" in seconds")

In [ ]:
prediction_train = pd.DataFrame(train_set, columns=['global_alighn_pred'])
prediction_test = pd.DataFrame(test_set, columns=['global_alighn_pred'])
baseline_train = pd.DataFrame(data, columns=['global_align'])
baseline_test = pd.DataFrame(dataFrame, columns=['global_align'])
pred_labels,cluster_pred=map_data_to_clusters(prediction_train,prediction_test)
baseline_labels,cluster_base=map_data_to_clusters(baseline_train,baseline_test)

In [ ]:
nb_cluster(baseline_labels, pred_labels,prediction_test)

In [ ]:
#**********************************Clustering end**********************************

In [ ]:
#***********************************RESULT start************************************

In [ ]:
class DataSetInfo_test:
    def __init__(self,first_org_name,second_org_name,chunk_index_first_org,chunk_index_second_org,first_chunk_value,second_chunk_value,
                word_count_score,longest_seq,rank_distance,levenshtein_distance,central_star_distance):
        self.first_org_name = first_org_name
        self.second_org_name = second_org_name
        self.chunk_index_first_org = chunk_index_first_org
        self.chunk_index_second_org  = chunk_index_second_org
        self.first_chunk_value = first_chunk_value
        self.second_chunk_value = second_chunk_value
        #alignment_free
        self.word_count_score = word_count_score
        self.longest_seq = longest_seq 
        self.rank_distance = rank_distance 
        self.levenshtein_distance = levenshtein_distance
        self.central_star_distance=central_star_distance
        #other metrics
        self.mean = None 
        self.standart_dev = None
        self.signalToNoise = None

    def as_dict(self):
        return {'org1': self.first_org_name, 'org2': self.second_org_name, 'chunk_or1': self.first_chunk_value,
               'chunk_or2': self.second_chunk_value,
                'word_count_score':self.word_count_score,'longest_seq':self.longest_seq,
                'rank_distance':self.rank_distance,'levenshtein_distance':self.levenshtein_distance,
                'central_star_distance':self.central_star_distance,
               'mean': self.mean,'STD':self.standart_dev,'signalToNoise':self.signalToNoise}

In [ ]:
def chunk_preprocessing_test(org1Name,Org2Name,chunks_seq_virus1,chunks_seq_virus2,seq1
                                       ,seq2,gap_open_penalty,gap_extend_penalty,result): 
    
    word_count=word_neighborhood_count_score(seq1, seq2)
    central_star_distance=getCentralStarDistance(seq1, seq2)
    rank_distance=rankDistance(seq1, seq2)
    longest_substring=longestSubstring(seq1, seq2)
    leveshtein_distance=leveshteinDistance(seq1, seq2)
    
    
    info =  DataSetInfo_test(org1Name,Org2Name,chunks_seq_virus1,chunks_seq_virus2,seq1,seq2,
                        word_count,longest_substring,rank_distance,leveshtein_distance,central_star_distance)
    result.put(info)

In [ ]:
import threading, queue
  
def paraller_preprocessing_test(org1Name,Org2Name,chunks_seq_virus1,chunks_seq_virus2,gap_open_penalty,gap_extend_penalty):
    q = queue.Queue()
    threads = []
    for x in range(int(len(chunks_seq_virus1))):
        for y in range(int(len(chunks_seq_virus2))):
            seq1 = chunks_seq_virus1[x]
            seq2 = chunks_seq_virus2[y]
            t=threading.Thread(target=chunk_preprocessing_test, args=(org1Name,Org2Name,chunks_seq_virus1,chunks_seq_virus2,seq1,seq2,gap_open_penalty,gap_extend_penalty,q))
            threads.append(t)
            t.start()
     # Wait for all threads to finish
    for x in threads:
        x.join()
    result=list(q.queue)
    return result

In [ ]:
import time
start = time.time()
result=paraller_preprocessing("Covid","HIV",chunks_seq_virus_covid,chunks_seq_virus_hiv,-10,-1)
filename="data_Covid_HIV_real.csv"
result.to_csv(filename,index = False, header=True)
#tuberculosis
end = time.time()
print("Time for executing model")
print(end - start)
print(" in seconds")
data_hiv_covid=pd.read_csv("data_Covid_HIV_real.csv")

In [ ]:
#X_test_hc=test_set.copy()
# prediction_train = pd.DataFrame(train_set, columns=['global_alighn_pred'])
# prediction_test = pd.DataFrame(test_set, columns=['global_alighn_pred'])
X_test_hc = pd.concat([prediction_train ,prediction_test])

X_test_hc['clusters']=cluster_pred.predict(X_test_hc)
#X_test_hc['chunk_or1']=data_hiv_covid['chunk_or1']
#X_test_hc['chunk_or2']=data_hiv_covid['chunk_or2']

In [ ]:
X_test_hc=test_set.copy()
X_test_hc['clusters']=pred_labels
X_test_hc['chunk_or1']=dataFrame['chunk_or1']
X_test_hc['chunk_or2']=dataFrame['chunk_or2']

In [ ]:
#find metrics for clusters


In [ ]:
clust_num=14
print("големина на клъстер {}:{} примера".format(clust_num,len(X_test_hc[(X_test_hc['clusters']==clust_num)])))
print("максимален скор в клъстер {}:{} ".format(clust_num,X_test_hc[(X_test_hc['clusters']==clust_num)]['global_alighn_pred'].max()))
print("минимален скор в клъстер {}:{} ".format(clust_num,X_test_hc[(X_test_hc['clusters']==clust_num)]['global_alighn_pred'].min()))

In [ ]:
import pandas as pd
import numpy as np
#!pip install seaborn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
#plot data with seaborn
facet = sns.lmplot(data=X_test_hc, x='global_alighn_pred', y='rank_distance', hue='clusters', 
                   fit_reg=False, legend=True, legend_out=True)

In [ ]:
#test with cluster omega

In [ ]:
X_test_hc_r=X_test_hc[['clusters','chunk_or1','chunk_or2','global_alighn_pred']]

In [ ]:
testM=X_test_hc_r[X_test_hc_r['chunk_or1'].str.contains('TCTGACAAATTCACAGATGGTGTATGCCTATTTTGGAA', regex=False)]
testA=testM[testM['chunk_or2'].str.contains('TCGAGCTTGCTACAAGGGACTTTCCGCTGGGGAC', regex=False)]
testA

In [ ]:
testM['chunk_or2'][33265]

In [ ]:
test=X_test_hc[(X_test_hc.clusters==9)]
test1=test.sort_values('global_alighn_pred')
test1[10:20]

In [ ]:
test['chunk_or1'][15784]

In [ ]:
import time
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

start = time.time()
result=paraller_preprocessing("Covid","HIV",chunks_seq_virus_covid,chunks_seq_virus_hiv,-10,-1)
dataFrame = pd.DataFrame([x.as_dict() for x in result])
scaler=StandardScaler()
test_set=dataFrame.copy()
test_set=test_set.drop(['org1', 'org2','chunk_or1','chunk_or2','mean', 'STD','signalToNoise',], axis=1)
test_set=test_set.dropna()
test_set=test_set.astype('int32')
test_set_transform =scaler.fit_transform(test_set)
test_set['local_alighn_pred']=loaded_model.predict(test_set_transform)
test_set['global_alighn_gap_pred']=loaded_model_ga_gap.predict(test_set_transform)
test_set_transform =scaler.fit_transform(test_set)
test_set['global_alighn_pred']=loaded_model_ga.predict(test_set_transform)
end = time.time()
print("Time for executing model")
print(end - start)
print(" in seconds")

In [ ]:
from sklearn.cluster import KMeans
start = time.time()
prediction_train = pd.DataFrame(test_set, columns=['global_alighn_pred'])
cluster = KMeans(n_clusters=15, init='k-means++', random_state=0)
cluster.fit_predict(prediction_train)
end = time.time()
print("Time for executing model")
print(end - start)
print(" in seconds")

In [ ]:
#***********************************RESULT end************************************

In [ ]:
#convert to dataframe
import pandas as pd
df = pd.DataFrame([x.as_dict() for x in result])

In [ ]:
df.to_csv (r'C:\workspace\data_HIV_Tubercolosis.csv', index = False, header=True)
df

In [ ]:
print("**********************************HIV**********HIV************************")

In [ ]:
import threading, queue
  
def paraller_preprocessing(org1Name,chunks_seq_virus1,gap_open_penalty,gap_extend_penalty):
    q = queue.Queue()
    threads = []
    for x in range(int(len(chunks_seq_virus1))):
        seq1 = chunks_seq_virus1[x]
        t=threading.Thread(target=chunk_preprocessing, args=(org1Name,org1Name,chunks_seq_virus1,chunks_seq_virus1,seq1,seq1,gap_open_penalty,gap_extend_penalty,q))
        threads.append(t)
        t.start()
     # Wait for all threads to finish
    for x in threads:
        x.join()
    result=list(q.queue)
    
    

In [ ]:
import time
start = time.time()
result=paraller_preprocessing("HIV",chunks_seq_virus_hiv,-10,-1)
#tuberculosis
end = time.time()
print("Time for executing model")
print(end - start)
print(" in seconds")

In [ ]:
#convert to dataframe
import pandas as pd
df = pd.DataFrame([x.as_dict() for x in result])

In [ ]:
df.to_csv (r'C:\workspace\data_HIV_HIV.csv', index = False, header=True)
df